In [ ]:
pip install wandb torch torchvision numpy matplotlib tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import wandb
wandb.login()



<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shahi77 (shahi77-national-institute-of-technology-hamirpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import wandb

# Initialize a new WandB run
wandb.init(project="Assignment03(CS-661)-experiments")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transformations for dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load dataset
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 15.7MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 482kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.78MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.65MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, activation_fn):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_size = input_size

        for size in hidden_layers:
            layers.append(nn.Linear(prev_size, size))
            layers.append(activation_fn())  # Example: ReLU, Sigmoid, etc.
            prev_size = size

        layers.append(nn.Linear(prev_size, output_size))  # Output layer
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x.view(x.size(0), -1))  # Flatten input


In [ ]:
def train(model, train_loader, optimizer, loss_fn, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        accuracy = correct / total

        # Log metrics in WandB
        wandb.log({"Epoch": epoch + 1, "Loss": total_loss / len(train_loader), "Accuracy": accuracy})

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}")



In [ ]:
# Define model, loss function, and optimizer
model = NeuralNetwork(input_size=784, hidden_layers=[128, 64], output_size=10, activation_fn=nn.ReLU)
loss_fn = nn.CrossEntropyLoss()

optimizers = {
    "SGD": optim.SGD(model.parameters(), lr=0.01),
    "Momentum": optim.SGD(model.parameters(), lr=0.01, momentum=0.9),
    "Adam": optim.Adam(model.parameters(), lr=0.001),
    "RMSProp": optim.RMSprop(model.parameters(), lr=0.001)
}

# Run experiments with different optimizers
for opt_name, opt in optimizers.items():
    print(f"\nTraining with {opt_name}")
    wandb.init(project="deep-learning-experiments", name=f"optimizer-{opt_name}")
    train(model, train_loader, opt, loss_fn, epochs=5)
    wandb.finish()  # Finish logging this run



Training with SGD
Epoch [1/5], Loss: 1346.0727, Accuracy: 0.7964
Epoch [2/5], Loss: 584.4407, Accuracy: 0.9085
Epoch [3/5], Loss: 479.1311, Accuracy: 0.9242
Epoch [4/5], Loss: 401.1213, Accuracy: 0.9363
Epoch [5/5], Loss: 341.9966, Accuracy: 0.9468


Accuracy,▁▆▇██
Epoch,▁▃▅▆█
Loss,█▃▂▁▁
Accuracy,0.9468
Epoch,5
Loss,0.1824



Training with Momentum


Epoch [1/5], Loss: 404.6737, Accuracy: 0.9340
Epoch [2/5], Loss: 259.0298, Accuracy: 0.9564
Epoch [3/5], Loss: 201.9358, Accuracy: 0.9664
Epoch [4/5], Loss: 172.2437, Accuracy: 0.9707
Epoch [5/5], Loss: 149.9189, Accuracy: 0.9751


Accuracy,▁▅▇▇█
Epoch,▁▃▅▆█
Loss,█▄▂▂▁
Accuracy,0.97507
Epoch,5
Loss,0.07996


...

Epoch [1/5], Loss: 181.2112, Accuracy: 0.9695
Epoch [2/5], Loss: 157.7847, Accuracy: 0.9727
Epoch [3/5], Loss: 143.5668, Accuracy: 0.9755
Epoch [4/5], Loss: 127.2855, Accuracy: 0.9776
Epoch [5/5], Loss: 115.9070, Accuracy: 0.9795


Accuracy,▁▃▅▇█
Epoch,▁▃▅▆█
Loss,█▅▄▂▁
Accuracy,0.97953
Epoch,5
Loss,0.06182



Training with RMSProp


Epoch [1/5], Loss: 137.3821, Accuracy: 0.9779
Epoch [2/5], Loss: 106.7374, Accuracy: 0.9814
Epoch [3/5], Loss: 98.5893, Accuracy: 0.9833
Epoch [4/5], Loss: 94.6317, Accuracy: 0.9843
Epoch [5/5], Loss: 89.0706, Accuracy: 0.9851


Accuracy,▁▄▆▇█
Epoch,▁▃▅▆█
Loss,█▄▂▂▁
Accuracy,0.9851
Epoch,5
Loss,0.0475


In [ ]:
sweep_config = {
    "method": "grid",  # Can be "random" or "bayes" as well
    "metric": {"name": "Accuracy", "goal": "maximize"},
    "parameters": {
        "epochs": {"values": [5, 10]},
        "batch_size": {"values": [16, 32, 64]},
        "learning_rate": {"values": [0.001, 0.0001]},
        "optimizer": {"values": ["sgd", "adam", "rmsprop"]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="deep-learning-experiments")


Create sweep with ID: ntjma6uk
Sweep URL: https://wandb.ai/shahi77-national-institute-of-technology-hamirpur/deep-learning-experiments/sweeps/ntjma6uk


In [ ]:
def sweep_train():
    wandb.init()
    config = wandb.config

    # Create model and optimizer dynamically
    model = NeuralNetwork(input_size=784, hidden_layers=[128, 64], output_size=10, activation_fn=nn.ReLU)
    optimizer = optimizers[config.optimizer]

    train(model, train_loader, optimizer, loss_fn, epochs=config.epochs)
    wandb.finish()

wandb.agent(sweep_id, function=sweep_train, count=5)


wandb: Agent Starting Run: h486ofvm with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd


wandb: ERROR Run h486ofvm errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-10-77c769e1a2ca>", line 7, in sweep_train
wandb: ERROR     optimizer = optimizers[config.optimizer]
wandb: ERROR                 ~~~~~~~~~~^^^^^^^^^^^^^^^^^^
wandb: ERROR KeyError: 'sgd'
wandb: ERROR 
wandb: Agent Starting Run: 8vi9j3na with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam


wandb: ERROR Run 8vi9j3na errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-10-77c769e1a2ca>", line 7, in sweep_train
wandb: ERROR     optimizer = optimizers[config.optimizer]
wandb: ERROR                 ~~~~~~~~~~^^^^^^^^^^^^^^^^^^
wandb: ERROR KeyError: 'adam'
wandb: ERROR 
wandb: Agent Starting Run: 0mwgycw2 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	optimizer: rmsprop


wandb: ERROR Run 0mwgycw2 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-10-77c769e1a2ca>", line 7, in sweep_train
wandb: ERROR     optimizer = optimizers[config.optimizer]
wandb: ERROR                 ~~~~~~~~~~^^^^^^^^^^^^^^^^^^
wandb: ERROR KeyError: 'rmsprop'
wandb: ERROR 
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


 Implement the experiments submitted  in as assignment 2 with the various gradient descent with different optimization functions.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Custom activation functions
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class ESwish(nn.Module):
    def __init__(self, beta=1.5):
        super().__init__()
        self.beta = beta

    def forward(self, x):
        return self.beta * x * torch.sigmoid(x)

activations = {
    "Sigmoid": nn.Sigmoid(),
    "Tanh": nn.Tanh(),
    "Mish": nn.Mish(),
    "ReLU": nn.ReLU(),
    "LeakyReLU": nn.LeakyReLU(0.01),
    "PReLU": nn.PReLU(),
    "ELU": nn.ELU(),
    "Softmax": nn.Softmax(dim=1),
    "Softplus": nn.Softplus(),
    "Swish": Swish(),
    "ESwish": ESwish(),
    "GELU": nn.GELU(),
    "SELU": nn.SELU()
}


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, activation_fn):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_size = input_size

        for size in hidden_layers:
            layers.append(nn.Linear(prev_size, size))
            layers.append(activation_fn)  # Dynamically assign activation function
            prev_size = size

        layers.append(nn.Linear(prev_size, output_size))  # Output layer
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x.view(x.size(0), -1))  # Flatten input


In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Common transformation for all datasets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to (-1,1)
])

datasets_dict = {
    "MNIST": datasets.MNIST(root="./data", train=True, transform=transform, download=True),
    "Fashion-MNIST": datasets.FashionMNIST(root="./data", train=True, transform=transform, download=True),
    "CIFAR-10": datasets.CIFAR10(root="./data", train=True, transform=transform, download=True),
    "CIFAR-100": datasets.CIFAR100(root="./data", train=True, transform=transform, download=True)
}

test_datasets_dict = {
    "MNIST": datasets.MNIST(root="./data", train=False, transform=transform, download=True),
    "Fashion-MNIST": datasets.FashionMNIST(root="./data", train=False, transform=transform, download=True),
    "CIFAR-10": datasets.CIFAR10(root="./data", train=False, transform=transform, download=True),
    "CIFAR-100": datasets.CIFAR100(root="./data", train=False, transform=transform, download=True)
}

# DataLoaders
def get_dataloader(dataset_name, batch_size=64):
    return DataLoader(datasets_dict[dataset_name], batch_size=batch_size, shuffle=True), \
           DataLoader(test_datasets_dict[dataset_name], batch_size=batch_size, shuffle=False)


100%|██████████| 26.4M/26.4M [00:01<00:00, 13.3MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 208kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.84MB/s]


...

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 170M/170M [00:03<00:00, 48.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


100%|██████████| 169M/169M [00:03<00:00, 46.4MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def train_model(model, train_loader, optimizer, loss_fn, epochs=10, activation_name="ReLU", dataset_name="MNIST"):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        accuracy = correct / total

        # Log results to WandB
        wandb.log({
            "Epoch": epoch + 1,
            "Loss": total_loss / len(train_loader),
            "Accuracy": accuracy,
            "Activation Function": activation_name,
            "Dataset": dataset_name
        })

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}")



In [ ]:
for dataset_name in ["MNIST", "Fashion-MNIST", "CIFAR-10", "CIFAR-100"]:
    train_loader, test_loader = get_dataloader(dataset_name)

    for act_name, act_func in activations.items():
        print(f"\nTraining on {dataset_name} with Activation: {act_name}")

        # Initialize WandB
        wandb.init(project="activation-functions-analysis", name=f"{dataset_name}-{act_name}")

        model = NeuralNetwork(input_size=784, hidden_layers=[128, 64], output_size=10, activation_fn=act_func)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()

        train_model(model, train_loader, optimizer, loss_fn, epochs=5, activation_name=act_name, dataset_name=dataset_name)

        wandb.finish()  # Finish logging



Training on MNIST with Activation: Sigmoid


Epoch [1/5], Loss: 611.7837, Accuracy: 0.8399
Epoch [2/5], Loss: 203.6508, Accuracy: 0.9383
Epoch [3/5], Loss: 142.4783, Accuracy: 0.9556
Epoch [4/5], Loss: 110.4503, Accuracy: 0.9652
Epoch [5/5], Loss: 91.9628, Accuracy: 0.9717


Accuracy,▁▆▇██
Epoch,▁▃▅▆█
Loss,█▃▂▁▁
Accuracy,0.97165
Activation Function,Sigmoid
Dataset,MNIST
Epoch,5
Loss,0.09804



Training on MNIST with Activation: Tanh


Epoch [1/5], Loss: 322.4641, Accuracy: 0.9040
Epoch [2/5], Loss: 149.1520, Accuracy: 0.9533
Epoch [3/5], Loss: 122.0153, Accuracy: 0.9610
Epoch [4/5], Loss: 106.0558, Accuracy: 0.9654
Epoch [5/5], Loss: 95.0870, Accuracy: 0.9687


...


Training on Fashion-MNIST with Activation: SELU


Epoch [1/5], Loss: 468.3474, Accuracy: 0.8175
Epoch [2/5], Loss: 358.7686, Accuracy: 0.8598
Epoch [3/5], Loss: 324.1862, Accuracy: 0.8721
Epoch [4/5], Loss: 300.5601, Accuracy: 0.8807
Epoch [5/5], Loss: 282.9461, Accuracy: 0.8880


Accuracy,▁▅▆▇█
Epoch,▁▃▅▆█
Loss,█▄▃▂▁
Accuracy,0.88803
Activation Function,SELU
Dataset,Fashion-MNIST
Epoch,5
Loss,0.30165



Training on CIFAR-10 with Activation: Sigmoid


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x3072 and 784x128)

In [ ]:
input_sizes = {
    "MNIST": 784,  # 28x28 grayscale
    "Fashion-MNIST": 784,  # 28x28 grayscale
    "CIFAR-10": 3072,  # 32x32x3 (color)
    "CIFAR-100": 3072  # 32x32x3 (color)
}

for dataset_name in ["MNIST", "Fashion-MNIST", "CIFAR-10", "CIFAR-100"]:
    train_loader, test_loader = get_dataloader(dataset_name)

    for act_name, act_func in activations.items():
        print(f"\nTraining on {dataset_name} with Activation: {act_name}")

        # Initialize WandB
        wandb.init(project="activation-functions-analysis", name=f"{dataset_name}-{act_name}")

        model = NeuralNetwork(
            input_size=input_sizes[dataset_name],  # Dynamically adjust input size
            hidden_layers=[128, 64],
            output_size=10,
            activation_fn=act_func
        )

        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()

        train_model(model, train_loader, optimizer, loss_fn, epochs=5, activation_name=act_name, dataset_name=dataset_name)

        wandb.finish()  # Finish logging



Training on MNIST with Activation: Sigmoid


Epoch [1/5], Loss: 600.7933, Accuracy: 0.8392
Epoch [2/5], Loss: 205.3074, Accuracy: 0.9378
Epoch [3/5], Loss: 145.2399, Accuracy: 0.9552
Epoch [4/5], Loss: 112.8483, Accuracy: 0.9646
Epoch [5/5], Loss: 92.2043, Accuracy: 0.9718


Accuracy,▁▆▇███
Epoch,▁▁▃▅▆█
Loss,█▄▂▁▁▁
Accuracy,0.97183
Activation Function,Sigmoid
Dataset,MNIST
Epoch,5
Loss,0.0983



Training on MNIST with Activation: Tanh


Epoch [1/5], Loss: 325.7528, Accuracy: 0.9020
Epoch [2/5], Loss: 151.1027, Accuracy: 0.9504
Epoch [3/5], Loss: 114.2542, Accuracy: 0.9625
Epoch [4/5], Loss: 102.2772, Accuracy: 0.9664
Epoch [5/5], Loss: 96.5635, Accuracy: 0.9674


...


Training on CIFAR-10 with Activation: SELU


Epoch [1/5], Loss: 1289.2967, Accuracy: 0.4129
Epoch [2/5], Loss: 1123.8529, Accuracy: 0.4890
Epoch [3/5], Loss: 1047.7950, Accuracy: 0.5255
Epoch [4/5], Loss: 986.2791, Accuracy: 0.5494
Epoch [5/5], Loss: 942.4435, Accuracy: 0.5711


Accuracy,▁▄▆▇█
Epoch,▁▃▅▆█
Loss,█▅▃▂▁
Accuracy,0.57112
Activation Function,SELU
Dataset,CIFAR-10
Epoch,5
Loss,1.20517



Training on CIFAR-100 with Activation: Sigmoid


IndexError: Target 45 is out of bounds.

In [ ]:
from torchvision import models

architectures = {
    "ResNet": models.resnet18(pretrained=True),
    "LeNet": models.alexnet(pretrained=True),
    "MobileNet": models.mobilenet_v2(pretrained=True),
    "AlexNet": models.alexnet(pretrained=True)
}

def modify_architecture(model, activation_fn):
    """Replaces all activation functions in a model with the specified activation function."""
    for name, module in model.named_modules():
        if isinstance(module, (nn.ReLU, nn.LeakyReLU, nn.GELU)):  # Include other activations
            parent_name = name.rsplit('.', 1)[0] if '.' in name else ''
            if parent_name:
                parent_module = dict(model.named_modules())[parent_name]
                setattr(parent_module, name.split('.')[-1], activation_fn())
            else:
                setattr(model, name, activation_fn())  # For top-level modules
    return model


for arch_name, arch_model in architectures.items():
    for act_name, act_func in activations.items():
        print(f"\nTraining {arch_name} with Activation: {act_name}")

        wandb.init(project="deep-architectures-analysis", name=f"{arch_name}-{act_name}")

        # Modify activation functions
        modified_model = modify_architecture(arch_model, act_func)

        # Adjust output layer for CIFAR-10 (10 classes instead of 1000)
        if isinstance(modified_model, models.ResNet):
            modified_model.fc = nn.Linear(modified_model.fc.in_features, 10)
        elif isinstance(modified_model, models.AlexNet) or isinstance(modified_model, models.MobileNetV2):
            modified_model.classifier[-1] = nn.Linear(modified_model.classifier[-1].in_features, 10)

        train_model(modified_model, train_loader, torch.optim.Adam(modified_model.parameters(), lr=0.001), nn.CrossEntropyLoss(), epochs=5, activation_name=act_name, dataset_name="CIFAR-10")

        wandb.finish()




Training ResNet with Activation: Sigmoid


TypeError: Sigmoid.forward() missing 1 required positional argument: 'input'